### Fine tune with axolotl

In [ ]:
# env
!git clone https://github.com/OpenAccess-AI-Collective/axolotl
%cd axolotl

!pip3 install packaging
!pip3 install -e '.[flash-attn,deepspeed]'
!pip3 install -U git+https://github.com/huggingface/peft.git


In [ ]:
# config
import os
# os.environ["NCCL_P2P_DISABLE"] = "1"
# os.environ["NCCL_BLOCKING_WAIT"] = "0"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
!wandb login XXXX
!huggingface-cli login --token XXXX
!accelerate launch  --multi_gpu --num_processes=4 -m axolotl.cli.train  ./qlora.yaml

### Save and upload

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
import torch

# Load the entire model on the GPU 0
device_map = {"": 0}
!huggingface-cli login --token XXXX


model_name = "meta-llama/Llama-2-13b-chat-hf"
new_model = "llama2_13b_v26_50k"


base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # left for mistral

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)